In [3]:
import pandas as pd
import re
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder


In [9]:

def get_auc(test_question):
    label_encoder = LabelEncoder()
    test_question['label_encoded'] = label_encoder.fit_transform(test_question['label'])
    test_question['random_answer_encoded'] = label_encoder.transform(test_question['random_answer'])
    auc_score = roc_auc_score(test_question['label_encoded'], test_question['random_answer_encoded'])
    return auc_score

def extract_answer(text):
    pattern = r"(True|False|Don't know)"
    matches = re.findall(pattern, text, re.IGNORECASE)
    return matches


In [17]:
RESPONSE_PATH = "../../../data/analysis_results/Llama_2_7b_Chat_GPTQ_prompt_based_response.csv"


In [18]:
response_df = pd.read_csv(RESPONSE_PATH)
response_df['llm_answer'] = response_df['llm_answer'].apply(extract_answer)
# response_df = response_df.explode("llm_answer")

response_df.head()
print(response_df.shape)

(609, 3)


In [8]:
response_df.llm_answer.unique()

array(['False', "Don't know", 'True', 'true'], dtype=object)

In [168]:
response_df_uncertain_response = response_df[response_df.llm_answer == "Don't know"]
response_df_certain_response = response_df[response_df.llm_answer != "Don't know"]
response_df_certain_response.loc[:, "llm_answer"] = response_df_certain_response.llm_answer.astype(bool)



/var/folders/p1/h56gxdhs5vgb0ztp7h4z606h0000gn/T/ipykernel_25408/271416556.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  response_df_certain_response.loc[:, "llm_answer"] = response_df_certain_response.llm_answer.astype(bool)
/var/folders/p1/h56gxdhs5vgb0ztp7h4z606h0000gn/T/ipykernel_25408/271416556.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  response_df_certain_response.loc[:, "llm_answer"] = response_df_certain_response.llm_answer.astype(bool)


In [169]:
total_certain_response = response_df_certain_response.shape[0]
total_uncertain_response = response_df_uncertain_response.shape[0]
total_response = response_df.shape[0]

correct_response = response_df_certain_response[response_df_certain_response.label == response_df_certain_response.llm_answer].shape[0]
incorrect_response = response_df_certain_response[response_df_certain_response.label != response_df_certain_response.llm_answer].shape[0]

true_response = correct_response/total_response
false_response = incorrect_response/total_response
uncertainty = total_uncertain_response/total_response


print("True response = ",true_response)
print("False response = ",false_response)
print("Uncertainty = ",uncertainty)


True response =  0.9507575757575758
False response =  0.04734848484848485
Uncertainty =  0.001893939393939394


In [170]:
response_df_certain_response

,question,label,llm_answer
0,Sandhoff Disease associates Gene HEXB,True,True
1,Liver carcinoma associates Gene TP53,True,True
2,Malignant neoplasm of breast associates Gene TP53,True,True
3,Li-Fraumeni Syndrome associates Gene TP53,True,True
4,Sarcoma associates Gene TP53,True,True
...,...,...,...
523,Disease ontology identifier for Norrie disease...,False,True
524,Disease ontology identifier for infantile myof...,False,True
525,Disease ontology identifier for otospondylomeg...,False,True
526,Disease ontology identifier for Pelizaeus-Merz...,False,True


In [171]:
response_df

,question,label,llm_answer
0,Sandhoff Disease associates Gene HEXB,True,False
1,Liver carcinoma associates Gene TP53,True,True
2,Malignant neoplasm of breast associates Gene TP53,True,True
3,Li-Fraumeni Syndrome associates Gene TP53,True,True
4,Sarcoma associates Gene TP53,True,True
...,...,...,...
523,Disease ontology identifier for Norrie disease...,False,True
524,Disease ontology identifier for infantile myof...,False,True
525,Disease ontology identifier for otospondylomeg...,False,False
526,Disease ontology identifier for Pelizaeus-Merz...,False,True


In [19]:
response_df.loc[:, "answer_count"] = response_df.llm_answer.apply(lambda x:len(x))
response_df.answer_count.max()

3

In [22]:
response_df[response_df.answer_count > 1]

,question,label,llm_answer,answer_count
24,Variant rs4402960 associates Diabetes Mellitus...,True,"[True, False, Don't know]",3
44,Thrombasthenia associates Gene ITGB3,True,"[True, False, Don't know]",3
50,Deficiency of pyruvate kinase associates Gene ...,True,"[True, False, Don't know]",3
90,Variant rs13266634 associates Diabetes Mellitu...,True,"[True, False, Don't know]",3
116,Alpha-Sarcoglycanopathies associates Gene SGCA,True,"[True, False, Don't know]",3
125,"HEMOCHROMATOSIS, TYPE 4 associates Gene SLC40A1",True,"[True, False, Don't know]",3
151,"MATURITY-ONSET DIABETES OF THE YOUNG, TYPE 3 (...",True,"[True, False, Don't know]",3
190,"Hyperkeratosis, Epidermolytic associates Gene ...",True,"[True, False, Don't know]",3
193,Rapp-Hodgkin syndrome is a autosomal dominant ...,True,"[True, False, Don't know]",3
201,Deficiency of butyryl-CoA dehydrogenase associ...,True,"[True, False, Don't know]",3


In [23]:
response_df = pd.read_csv(RESPONSE_PATH)

In [24]:
response_df.iloc[577]

question      Congenital bilateral aplasia of vas deferens a...
label                                                      True
llm_answer          {answer : <True> or <False> or <Don't know>
Name: 577, dtype: object